In [ ]:
!pip install -r napari_interactive_example_requirements.txt

# Initialize napari

In [1]:
import napari
import numpy as np
import pandas as pd

from IPython.display import display
from skimage.io import imread
from skimage.measure import regionprops_table
from laptrack import LapTrack
from laptrack.data_conversion import convert_dataframe_to_coords, convert_tree_to_dataframe

In [2]:
images = imread("interactive_example_data/demo_image.tif")
labels = np.load("interactive_example_data/labels3.npy")

In [3]:
viewer = napari.Viewer()
viewer.add_image(images, name="images")
viewer.add_labels(labels, name="labels")

<Labels layer 'labels' at 0x1b60fbe80>

# Calculate properties of the segmentation

In [4]:
def calc_frame_regionprops(labels):  # noqa: E302
    dfs = []
    for frame in range(labels.shape[0]):
        df = pd.DataFrame(
            regionprops_table(labels[frame], properties=["label", "area", "centroid"])
        )
        df["frame"] = frame
        dfs.append(df)
    return pd.concat(dfs)
regionprops_df = calc_frame_regionprops(labels)
display(regionprops_df)

,label,area,centroid-0,centroid-1,frame
0,2,1149,21.095735,51.044386,0
1,3,676,13.699704,73.125740,0
2,4,900,13.463333,99.415556,0
3,5,706,14.866856,133.256374,0
4,6,542,9.046125,20.151292,0
...,...,...,...,...,...
34,36,905,65.306077,10.375691,9
35,37,110,14.627273,141.709091,9
36,38,741,33.570850,126.275304,9
37,39,158,133.164557,139.373418,9


# Tracking by LapTrack

## Creating data

In [5]:
_coords = convert_dataframe_to_coords(
    regionprops_df, ["centroid-0", "centroid-1", "label"]
)
coords = [c[:, :-1] for c in _coords]
coord_labels = [c[:, -1] for c in _coords]

## Tracking

In [6]:
lt = LapTrack(splitting_cost_cutoff=20**2)
tree = lt.predict(coords)
tracked_df, _, _ = convert_tree_to_dataframe(tree)

## Adding the tracked data to the viewer

In [8]:
_regionprops_df = regionprops_df.set_index(["frame", "label"])
for (frame, index), row in tracked_df.iterrows():
    label = coord_labels[frame][index]
    _regionprops_df.loc[(frame, label), "track_id"] = row["track_id"] 
track_label_image = np.zeros_like(labels)
for (frame, label), row in _regionprops_df.iterrows():
    track_label_image[frame][labels[frame] == label] = row["track_id"] + 1

In [9]:
viewer.layers["labels"].visible = False
viewer.add_labels(track_label_image)

<Labels layer 'track_label_image' at 0x1bd1a2a40>

# Manual correction

add points for the cells validated maually (emurated)

In [10]:
manual_corrected = np.load("interactive_example_data/manual_corrected.npy")
viewer.add_points(manual_corrected, name="manually_validated_tracks")

<Points layer 'manually_validated_tracks' at 0x1bd1a24d0>

In [11]:
manual_corrected = viewer.layers["manually_validated_tracks"].data.astype(np.int16)
# you can also redraw the labels
new_labels = viewer.layers["track_label_image"].data
# get label values at the placed points
validated_track_labels = new_labels[tuple(manual_corrected.T)]
validated_frames = manual_corrected[:, 0]

In [12]:
validated_points=np.array(list(zip(validated_frames, validated_track_labels)))
validated_points=validated_points[np.argsort(validated_points[:,0])]
validated_ind_pairs=[
    set(((frame1,label1),(frame2,label2))) for ((frame1,label1),(frame2,label2)) 
        in zip(validated_points[:-1],validated_points[1:])
]
validated_ind_pairs

[{(0, 34), (1, 34)},
 {(1, 34), (2, 37)},
 {(2, 37), (3, 37)},
 {(3, 37), (4, 37)},
 {(4, 37), (5, 37)},
 {(5, 37), (6, 37)},
 {(6, 37), (7, 37)},
 {(7, 37), (8, 37)},
 {(8, 37), (9, 37)}]

# Second tracking preserving manually corrected data

In [13]:
new_regionprops_df = calc_frame_regionprops(new_labels).set_index(["frame", "label"]).reset_index()
new_coords = convert_dataframe_to_coords(
    new_regionprops_df,
    ["centroid-0", "centroid-1", "frame", "label"],
)
coord_labels = [c[:, -1] for c in new_coords]
new_regionprops_df.loc[[33,63,95]]

,frame,label,area,centroid-0,centroid-1
33,0,34,621,23.745572,14.080515
63,1,34,576,30.123264,14.315972
95,2,37,537,26.126629,26.819367


pairs of the coordinates of the validated points

In [14]:
def manual_corrected_aware_dist(c1,c2):
    *coords1, frame1, label1 = c1
    *coords2, frame2, label2 = c2
    pair={( int(frame1),int(label1),), (int(frame2),int(label2))}
    if pair in validated_ind_pairs:
        return 0.0001
    else:
        return np.linalg.norm(np.array(coords1)-np.array(coords2))**2

lt = LapTrack(splitting_cost_cutoff=20**2,
              track_dist_metric=manual_corrected_aware_dist,
              splitting_dist_metric=manual_corrected_aware_dist,)
new_tree = lt.predict(new_coords)
new_tracked_df, _, _ = convert_tree_to_dataframe(new_tree)

In [15]:
new_track_label_image = np.zeros_like(new_labels)
for (frame, ind), row in new_tracked_df.iterrows():
    label=new_labels[frame] == coord_labels[frame][ind]
    new_track_label_image[frame][label] = row["tree_id"] + 1
viewer.layers["track_label_image"].visible = False
viewer.add_labels(new_track_label_image)

KeyError: 'clone_id'